In [1]:
#Imports
import os
import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
#Calculate Correlation
from scipy.stats import pearsonr
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

# Input location of parameter

In [2]:
analysis = 'kernel_ridge_regression_outliers_included'
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    data_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/addbs_vta_to_memory_net_maxima/linear_regression_addbs_vta_to_memory_net_maxima/prepared_data_sbc_consolidated.csv'
    out_dir = os.path.join(os.path.dirname(data_path), f'{analysis}')
    #out_dir = r'path to out dir here'
    
    print('I have set pathnames in the Mac style')
    print('I will save to :', out_dir)
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    data_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\addbs_vta_to_memory_net_maxima\linear_regression_addbs_vta_to_memory_net_maxima\prepared_data.csv'
    out_dir = rf'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\addbs_vta_to_memory_net_maxima\linear_regression_addbs_vta_to_memory_net_maxima\{analysis}'
    #out_dir = r'path to out dir here'

    print('I have set pathnames in the Windows style')

I have set pathnames in the Mac style
I will save to : /Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/addbs_vta_to_memory_net_maxima/linear_regression_addbs_vta_to_memory_net_maxima/kernel_ridge_regression_outliers_included


## Import pre-prepared dataframe

In [3]:
data_df = pd.read_csv(data_path)
data_df.tail(3)

,Unnamed: 0,% Change from baseline (ADAS-Cog11),Occ. med. L,Par. Lat. R,PFC Mid. Vent.,Cereb. IX L,Temp. Lat. L,Thal. Ant. Med. L,Front. sup. L,Cereb. IX R,Front. Sup. R,Occ. Med. R,Par. Lat. L,Corp. Call. R,Thal. Ant. Med. R,Temp. Lat. R,SBC
48,sub-1201uvatuseeducompoundufMRIuefieldubl_spac...,-7.142857,-4.246929,7.334911,27.745649,9.548862,14.586241,21.880682,5.883314,9.201521,2.955958,-5.607180,8.230703,0.666332,20.275417,15.967279,28.907301
49,sub-1202uvatuseeducompoundufMRIuefieldubl_spac...,-2.255639,-4.623723,8.189873,25.385062,9.450783,13.817418,16.817421,4.044289,9.237000,2.488460,-5.798545,8.237633,-0.186486,17.285401,15.645035,28.123434
50,sub-1203uvatuseeducompoundufMRIuefieldubl_spac...,-73.333333,-3.892837,9.661470,42.494464,12.493882,14.889942,46.757305,16.594608,13.320380,10.304421,-6.019783,12.975199,5.107108,46.990210,18.456445,40.971289


## Clean the dataframe

In [4]:
data_df = data_df.reset_index(drop=True)
try:
    data_df.pop('Unnamed: 0')
except:
    print('none to pop')
data_df = data_df.rename(columns={'% Change from baseline (ADAS-Cog11)': 'percent_change_adascog11'})#, '07_default': 'default', '02_somatomotor_seed': 'somatomotor', '03_dorsal_attention': 'dorsal', '01_visual_seed': 'visual', '04_ventral_attention': 'ventral', '05_limbic': 'limbic', '06_frontoparietal': 'frontoparietal'})
if np.sum(np.isnan(data_df)).any():
    data_df.dropna(inplace=True) #data_df.fillna(method='bfill', inplace=True)
    data_df.reset_index(inplace=True, drop=True)
display(data_df)


,percent_change_adascog11,Occ. med. L,Par. Lat. R,PFC Mid. Vent.,Cereb. IX L,Temp. Lat. L,Thal. Ant. Med. L,Front. sup. L,Cereb. IX R,Front. Sup. R,Occ. Med. R,Par. Lat. L,Corp. Call. R,Thal. Ant. Med. R,Temp. Lat. R,SBC
0,-21.428571,-5.782694,4.012973,20.651449,5.391541,15.079160,-3.093645,-0.304522,5.669152,-3.217608,-6.192463,4.720244,-1.734131,-3.458998,14.687544,17.936317
1,-36.363636,-9.392699,4.673912,26.615519,7.156203,17.582581,-2.200953,1.155054,7.562589,-3.751051,-9.292780,6.126371,-4.735250,-2.484542,17.523064,17.806708
2,-78.947368,-7.329420,6.605479,27.133585,6.858777,18.577697,2.253035,3.756483,7.218829,-0.336902,-7.971693,8.035506,-0.202378,1.870699,18.714626,22.211831
3,-129.411765,-3.227850,4.932552,16.298642,6.628872,11.120452,1.138491,-1.054035,6.807925,-1.683538,-4.048880,4.588551,-0.728670,0.914263,11.343452,17.446499
4,-10.526316,-5.443074,7.019019,11.087058,8.493880,1.855083,4.932595,-0.312372,9.230444,0.785067,-3.885054,5.515120,-2.320399,5.400880,2.787548,18.317946
5,-38.461538,-5.624823,6.965686,24.957602,6.529833,17.529284,5.100116,3.929140,6.661270,0.609743,-6.751083,8.230193,0.104832,5.565769,18.362639,24.544471
6,-15.384615,-5.506084,7.000901,12.254034,8.295950,4.471971,4.168145,-0.852396,9.208154,-0.700660,-4.247400,5.654617,-1.808257,4.641854,5.575752,18.761324
7,-29.166667,-4.761093,8.705312,13.476399,10.277352,4.079299,8.224864,-0.284511,10.386122,1.729162,-3.395050,5.977413,-0.677367,8.599857,5.809259,20.281472
8,-30.434783,-7.625032,8.393050,35.954402,9.140423,19.387844,16.936647,9.377636,9.566475,3.673179,-8.935371,11.119537,1.550540,17.973579,21.590925,30.031877
9,-84.615385,-3.652154,8.262432,28.813812,7.002018,14.115958,27.545906,8.892986,6.980448,5.330480,-5.988235,9.641565,2.126186,30.548233,17.252100,32.240269


In [ ]:
from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled = scaler.fit_transform(data_df)
# data_df = pd.DataFrame(scaled)
data_df=(data_df-data_df.mean())/data_df.std()


In [8]:
#Choose specific columns to assess
new_df = data_df.copy()
# columns_of_interest = [
#     'SBC', 'Thal. Ant. Med. L', 'PFC Mid. Vent.'
    
# ]
# columns_to_extract = ['percent_change_adascog11'] + columns_of_interest
# new_df = new_df.loc[:, columns_to_extract]
# data_df = new_df
data_df.head(2)

,percent_change_adascog11,Occ. med. L,Par. Lat. R,PFC Mid. Vent.,Cereb. IX L,Temp. Lat. L,Thal. Ant. Med. L,Front. sup. L,Cereb. IX R,Front. Sup. R,Occ. Med. R,Par. Lat. L,Corp. Call. R,Thal. Ant. Med. R,Temp. Lat. R,SBC
0,-21.428571,-5.782694,4.012973,20.651449,5.391541,15.079160,-3.093645,-0.304522,5.669152,-3.217608,-6.192463,4.720244,-1.734131,-3.458998,14.687544,17.936317
1,-36.363636,-9.392699,4.673912,26.615519,7.156203,17.582581,-2.200953,1.155054,7.562589,-3.751051,-9.292780,6.126371,-4.735250,-2.484542,17.523064,17.806708


## Kernels

## Run a LOOC On a Kernel Ridge Regression

In [13]:
import pandas as pd
import numpy as np
from sklearn.kernel_ridge import KernelRidge

def kernel_ridge_loo(df:pd.DataFrame, target:str, alpha:float, gamma:float, degree:int, kernel:str):
    y_pred = []
    for i in range(0, len(df[target].values.tolist())):
        # splitting dataframe into train and test set
        df_test = df.loc[[i]]
        df_train = df.drop(df.index[i])
        
        X_train = df_train.drop(target, axis=1)
        y_train = df_train[target]
        X_test = df_test.drop(target, axis=1)
        y_test = df_test[target]
        
        model = KernelRidge(kernel=kernel, alpha=alpha, gamma=gamma, degree=degree)
        model.fit(X_train, y_train)
        y_pred.append(model.predict(X_test)[0])
        
    outcome_dictionary = {}
    outcome_dictionary['actuals'] = df[target].values.tolist()
    outcome_dictionary['predictions'] = y_pred
    try:
        outcome_dictionary['loocv_prediction_r'], outcome_dictionary['loocv_prediction_p'] = pearsonr(outcome_dictionary['actuals'], outcome_dictionary['predictions'])
        outcome_dictionary['loocv_prediction_r2'] = outcome_dictionary['loocv_prediction_r']**2
        outcome_dictionary['loocv_mean_squared_error'] = np.sum( np.square( (outcome_dictionary['actuals'] - outcome_dictionary['predictions']))) / len(outcome_dictionary['predictions'])
        outcome_dictionary['loocv_root_mean_squared_error'] = np.sqrt(outcome_dictionary['loocv_mean_squared_error'])
        outcome_dictionary['lootcv_mean_absolute_error'] = np.sum(np.abs((outcome_dictionary['predictions'] - outcome_dictionary['actuals']))) / len(outcome_dictionary['predictions'])    
    except:
        print('Error generating outcome metrics')
    outcome_df = pd.DataFrame(outcome_dictionary)
    
    return outcome_df


In [45]:
kernels = ['additive_chi2', 'polynomial', 'poly', 'rbf', 'cosine', 'laplacian', 'precomputed', 'sigmoid', 'linear', 'chi2']
kernel = 'polynomial'
loo_results = kernel_ridge_loo(df=data_df, target='percent_change_adascog11', kernel=kernel, alpha=.01, gamma=.0001, degree=2)
# loo_df = pd.DataFrame(loo_results)
from scipy.stats import pearsonr

#Calculate metrics
loo_results.dropna(inplace=True)
loo_results['loocv_pearson_r'], loo_results['loocv_pearson_p'] = pearsonr(loo_results['predictions'], loo_results['actuals'], alternative='greater')
loo_results['loocv_r_squared'] = np.square(loo_results['loocv_pearson_r'])
_, loo_results['loocv_pearson_p_greater'] = pearsonr(loo_results['predictions'], loo_results['actuals'], alternative='greater')
_, loo_results['loocv_pearson_p_lesser'] = pearsonr(loo_results['predictions'], loo_results['actuals'], alternative='less')
_, loo_results['loocv_pearson_p_two-sided'] = pearsonr(loo_results['predictions'], loo_results['actuals'], alternative='two-sided')
loo_results['loocv_mse'] = (np.sum(np.square((loo_results['actuals']-loo_results['predictions'])))) / len(loo_results['actuals'])
loo_results['loocv_rmse'] = np.sqrt(loo_results['loocv_mse'])
loo_results['mean_absolute_error'] = np.sum(np.absolute(loo_results['predictions']-loo_results['actuals']))/len(loo_results['predictions'])

display(loo_results)

Error generating outcome metrics


,actuals,predictions,loocv_prediction_r,loocv_prediction_p,loocv_prediction_r2,loocv_pearson_r,loocv_pearson_p,loocv_r_squared,loocv_pearson_p_greater,loocv_pearson_p_lesser,loocv_pearson_p_two-sided,loocv_mse,loocv_rmse,mean_absolute_error
0,-21.428571,-64.946470,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643
1,-36.363636,-59.111885,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643
2,-78.947368,-37.052603,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643
3,-129.411765,-55.292326,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643
4,-10.526316,-23.775395,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643
5,-38.461538,-36.566315,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643
6,-15.384615,-30.425519,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643
7,-29.166667,-32.756026,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643
8,-30.434783,-24.976869,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643
9,-84.615385,9.199161,-0.043877,0.76223,0.001925,-0.043877,0.381115,0.001925,0.381115,0.618885,0.76223,3030.453738,55.049557,43.25643


In [34]:
#Save if desired
save=True
if save:
    if os.path.isdir(out_dir) != True:
        os.makedirs(out_dir)
        print('Made directory')
    try:
        loo_results.to_csv(os.path.join(out_dir, f'loocv_{kernel}_kernel_ridge_regression_{data_df.columns.values.tolist()[1:]}_statistically_significant_{loo_results["mean_absolute_error"].values.tolist[1]}.csv'))
    except:
        loo_results.to_csv(os.path.join(out_dir, f'ols_leave_one_out_full_formula.csv'))


print('Saved to: ' + out_dir)

Saved to: /Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/addbs_vta_to_memory_net_maxima/linear_regression_addbs_vta_to_memory_net_maxima/kernel_ridge_regression_outliers_included


## Call a Custom Kernel, Run a Ridge Regression, and Get Coefficients

In [44]:
# Define the list of kernel functions
def linear_kernel(X1, X2, **kwargs):
    """Linear kernel function"""
    return np.matmul(X1, X2.transpose())

def polynomial_kernel(x1, x2, degree, c=1, **kwargs):
    return (np.dot(x1, x2) + c)**degree

def rbf_kernel(x1, x2, gamma, **kwargs):
    return np.exp(-gamma * np.power(np.linalg.norm(x1 - x2), 2))

def gaussian_kernel(x1, x2, sigma, **kwargs):
    return np.exp(-np.power(np.linalg.norm(x1 - x2), 2) / (2 * sigma ** 2))

def sigmoid_kernel(x1, x2, k=1, c=1, **kwargs):
    return np.tanh(k * np.dot(x1, x2) + c)

def laplacian_kernel(x1, x2, gamma, **kwargs):
    return np.exp(-gamma * np.linalg.norm(x1 - x2))

def exponential_kernel(x1, x2, gamma, **kwargs):
    return np.exp(-gamma * np.linalg.norm(x1 - x2))

In [49]:
import pandas as pd
import numpy as np
from sklearn.kernel_ridge import KernelRidge
from scipy.stats import pearsonr
kernel_list = ["linear_kernel", "polynomial_kernel", "rbf_kernel", "gaussian_kernel", "sigmoid_kernel", "laplacian_kernel", "exponential_kernel"] #, "exp", "log", "tanh"]
#Iterate over the kernels of interest

result_list_kernel = []; results_list_r2 = []; results_list_r = []; results_list_p = []; results_list_mse = []; results_list_rmse = []; results_list_mae = []
for kernel_string in kernel_list:

    #Generate Kernelized data
    n = data_df.shape[0]
    kernel = np.zeros((n,n))
    Y = data_df['percent_change_adascog11']
    X = data_df.drop('percent_change_adascog11', axis=1)
    sigma = 1000
    for i in range(n):
        for j in range(n):
            kernel[i,j] = eval(kernel_string)(X.iloc[i], X.iloc[j], gamma=1, sigma=1, k=1, degree=2)#, sigma=1)# degree=1)
            
    #Train a ridge regression on kernelized data
    model = KernelRidge(kernel='precomputed')
    results = model.fit(kernel[:, :], Y[:])

    results_dict = {}
    # results_dict['predictions'] = np.dot(kernel,model.dual_coef_)
    # results_dict['observations'] = Y[:]
    result_list_kernel.append(kernel_string)
    results_list_r2.append(results.score(kernel[:, :], Y[:]))
    r, p = pearsonr(np.dot(kernel,model.dual_coef_), Y[:])
    results_list_r.append(r); results_list_p.append(p)
    results_list_mse.append((np.sum(np.square((Y[:]-np.dot(kernel,model.dual_coef_))))) / len(Y[:]))
    results_list_rmse.append(np.sqrt((np.sum(np.square((Y[:]-np.dot(kernel,model.dual_coef_))))) / len(Y[:])))
    results_list_mae.append(np.sum(np.absolute(np.dot(kernel,model.dual_coef_)-Y[:]))/len(np.dot(kernel,model.dual_coef_)))
results_dict['kernel'] = result_list_kernel
results_dict['r2'] = results_list_r2; results_dict['r'] = results_list_r; results_dict['p'] = results_list_p
results_dict['mse'] = results_list_mse; results_dict['rmse'] = results_list_rmse; results_dict['mae'] = results_list_mae

kernel_comparison_kernel_ridge_results = pd.DataFrame(results_dict)
display(kernel_comparison_kernel_ridge_results)

/usr/local/Caskroom/mambaforge/base/envs/nimlab_py310/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


,kernel,r2,r,p,mse,rmse,mae
0,linear_kernel,0.251676,0.502104,2.030207e-04,1750.185065,41.835213,33.693689
1,polynomial_kernel,0.996305,0.998249,1.301264e-60,8.642003,2.939728,1.677769
2,rbf_kernel,0.614807,1.000000,4.917348e-298,900.892187,30.014866,21.589489
3,gaussian_kernel,0.614834,1.000000,3.174085e-164,900.828855,30.013811,21.589035
4,sigmoid_kernel,0.064963,0.274029,5.414116e-02,2186.869669,46.763978,35.424464
5,laplacian_kernel,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
6,exponential_kernel,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049


In [50]:
#Save Results if desired
save=True
if save:
    if os.path.isdir(out_dir) != True:
        os.makedirs(out_dir)
        print('Made directory')
    kernel_comparison_kernel_ridge_results.to_csv(os.path.join(out_dir, 'kernel_comparison_kernel_ridge_results.csv'))


print('Saved to: ' + out_dir)

Saved to: /Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/addbs_vta_to_memory_net_maxima/linear_regression_addbs_vta_to_memory_net_maxima/kernel_ridge_regression_outliers_included


## Run Kernels Manually

In [40]:
n = data_df.shape[0]
kernel = np.zeros((n,n))
Y = data_df['percent_change_adascog11']
X = data_df.drop('percent_change_adascog11', axis=1)
sigma = 1000
for i in range(n):
    for j in range(n):
        kernel[i,j] = exponential_kernel(X.iloc[i], X.iloc[j], gamma=1, k=1, degree=1)#, sigma=1)# degree=1)
            

In [43]:
#Generate Kernelized data
n = data_df.shape[0]
kernel = np.zeros((n,n))
Y = data_df['percent_change_adascog11']
X = data_df.drop('percent_change_adascog11', axis=1)
sigma = 1000
for i in range(n):
    for j in range(n):
        kernel[i,j] = exponential_kernel(X.iloc[i], X.iloc[j], gamma=1, k=1, degree=1)#, sigma=1)# degree=1)
        
#Train a ridge regression on kernelized data
model = KernelRidge(kernel='precomputed')
results = model.fit(kernel[:, :], Y[:])

results_dict = {}
results_dict['predictions'] = np.dot(kernel,model.dual_coef_)
results_dict['observations'] = Y[:]
results_dict['r2'] = results.score(kernel[:, :], Y[:])
results_dict['pearson_r'], results_dict['pearson_p'] = pearsonr(results_dict['predictions'], results_dict['observations'])
results_dict['mse'] = (np.sum(np.square((results_dict['observations']-results_dict['predictions'])))) / len(results_dict['observations'])
results_dict['rmse'] = np.sqrt(results_dict['mse'])
results_dict['mae'] = np.sum(np.absolute(results_dict['predictions']-results_dict['observations']))/len(results_dict['predictions'])

manual_kernel_ridge_results = pd.DataFrame(results_dict)
display(manual_kernel_ridge_results)


,predictions,observations,r2,pearson_r,pearson_p,mse,rmse,mae
0,-10.818408,-21.428571,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
1,-18.182747,-36.363636,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
2,-39.493211,-78.947368,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
3,-64.730613,-129.411765,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
4,-5.309426,-10.526316,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
5,-19.316076,-38.461538,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
6,-7.726325,-15.384615,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
7,-14.587345,-29.166667,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
8,-15.249097,-30.434783,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049
9,-42.322178,-84.615385,0.615568,0.999965,1.659555e-101,899.111251,29.985184,21.564049


## Kernel Ridge Regression, K-Folds Cross Validation, Default Gaussian Kernel/

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from scipy.stats import pearsonr
import numpy as np

def kernel_ridge_kfold(df:pd.DataFrame, target:str, k:int, sigma:float):
    kf = KFold(n_splits=k, shuffle=True)
    y_pred = []
    y_test_all = []
    for train_index, test_index in kf.split(df):
        df_train, df_test = df.iloc[train_index], df.iloc[test_index]
        X_train = df_train.drop(target, axis=1)
        y_train = df_train[target]
        X_test = df_test.drop(target, axis=1)
        y_test = df_test[target]
        model = KernelRidge(kernel='rbf', gamma=1/(2*sigma**2))
        model.fit(X_train, y_train)
        y_pred.append(model.predict(X_test))
        y_test_all.append(y_test)
    y_test_all = np.concatenate(y_test_all)
    y_pred = np.concatenate(y_pred)
    R, P = pearsonr(y_pred, y_test_all)
    return R, P

R, P = kernel_ridge_kfold(df=data_df, target='percent_change_adascog11', k=2, sigma=100)#(data_df.shape[0]-1)/6)
results_df = pd.DataFrame({'R': R, 'P': P}, index=[0])
display(results_df)

save=False
if save:
    if os.path.isdir(out_dir) == False:
        os.mkdir(out_dir)
    results_df.to_csv(os.path.join(out_dir, 'kernel_ridge_regression_k_folds_results.csv'))


## Extract Coefficients from Ridge Regression

In [ ]:
def kernel_ridge(df:pd.DataFrame, target:str, sigma:float):
    y_pred = []
    y_test_all = []
    df_train = df.iloc[:, :]
    X_train = df_train.drop(target, axis=1)
    y_train = df_train[target]
    model = KernelRidge(kernel='rbf', gamma=1/(2*sigma**2))
    model.fit(X_train, y_train)
    print(model.predict(X_train))
    
    #Extract the coefficients
    kernel_space_coefficients = model.dual_coef_
    normal_coefficients = np.dot(X_train.transpose(), kernel_space_coefficients)
    return normal_coefficients, kernel_space_coefficients

coefficients, kernel_space_coefficients = kernel_ridge(df=data_df, target='percent_change_adascog11', sigma=(data_df.shape[0]-1)/6)


In [ ]:
coefficients_df = pd.DataFrame({'Coefficient Name': data_df.columns.to_list()[1:], 'Values': coefficients})
display(coefficients_df)

save=True
if save:
    if os.path.isdir(out_dir) == False:
        os.mkdir(out_dir)
    results_df.to_csv(os.path.join(out_dir, 'kernel_ridge_regression_coefficient_weight_results.csv'))


## Run initial model on all data

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

# Split dataframe into response and independent variables
X = data_df.iloc[:,1:].astype('float32')
y = data_df.iloc[:,0].astype('float32')

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a list of kernel functions
kernels = [tf.exp(-tf.reduce_sum(tf.square(X_train - X_train), axis=-1) / (2 * tf.square(0.1))),
           tf.matmul(X_train,tf.transpose(X_train)),
           tf.pow(tf.matmul(X_train,tf.transpose(X_train)),2),
           tf.pow(tf.matmul(X_train,tf.transpose(X_train)),3),
           tf.exp(tf.reduce_sum(X_train * X_train, axis=1)),
        #    tf.log(1+tf.reduce_sum(X_train * X_train, axis=1)),
           tf.tanh(tf.reduce_sum(X_train * X_train, axis=1))]
kernel_names = ["RBF-0.1", "Linear", "2nd degree polynomial", "3rd degree polynomial", "Exponential", "Logarithmic", "Hyperbolic tangent"]

# Create placeholders for the input data
X_ph = tf.compat.v1.placeholder(tf.float32, shape=(None, 2))
y_ph = tf.compat.v1.placeholder(tf.float32, shape=(None,))

# Create a dense layer
dense = tf.layers.Dense(units=1)

for kernel, kernel_name in zip(kernels, kernel_names):
    # Compute pairwise distances using the kernel trick
    kernel_output = kernel(X_ph, X_ph)
    kernel_output_train = kernel(X_ph, X_train)
    kernel_output_test = kernel(X_ph, X_test)

    # Pass the kernel output through the dense layer
    output = dense(kernel_output)
    output_train = dense(kernel_output_train)
    output_test = dense(kernel_output_test)

    # Define the loss function and optimizer
    loss = tf.losses.MeanAbsoluteError(y_ph, output)
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for _ in range(1000):
            _, train_loss = sess.run([train_op, loss], feed_dict={X_ph: X_train, y_ph: y_train})

        # Evaluate the model on the test set
        test_output = sess.run(output_test, feed_dict={X_ph: X_test})
        test_mse = tf.losses.mean_squared_error(y_test, test_output)
        print(f"Kernel: {kernel_name}, Test MSE: {test_mse}")

## Run LOOCV

In [ ]:
## Define Kernels

import tensorflow as tf
import numpy as np




In [ ]:
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import regularizers


def leave_one_out_cross_validation(df, kernels, kernel_names):
    # Split dataframe into response and independent variables
    X = df.iloc[:,1:].astype('float32')
    y = df.iloc[:,0].astype('float32')
    n = len(X)
    mae_list = []
    for i in range(n):
        X_test = X.iloc[i].values.reshape(1,-1)
        y_test = y.iloc[i]
        X_train = X.drop(i)
        y_train = y.drop(i)

        for kernel, kernel_name in zip(kernels, kernel_names):
            # Compute pairwise distances using the kernel trick
            if kernel_name == 'polynomial2':
                kernel_output = kernel(X_train, X_train, degree=2)
                kernel_output_test = kernel(X_test, X_train, degree=2)            
            if kernel_name == 'polynomial3':
                kernel_output = kernel(X_train, X_train, degree=3)
                kernel_output_test = kernel(X_test, X_train, degree=3)
            elif kernel_name == 'rbf':
                kernel_output = kernel(X_train, X_train, gamma=1/X_train.shape[1])
                kernel_output_test = kernel(X_test, X_train, gamma=1/X_train.shape[1])
            elif kernel_name == 'exp':
                kernel_output = kernel(X_train, X_train, gamma=1.0)
                kernel_output_test = kernel(X_test, X_train, gamma=1.0)
            elif kernel_name == 'log':
                kernel_output = kernel(X_train, X_train, alpha=1.0)
                kernel_output_test = kernel(X_test, X_train, alpha=1.0)
            elif kernel_name == 'tanh':
                kernel_output = kernel(X_train, X_train, alpha=1.0, beta=1.0)
                kernel_output_test = kernel(X_test, X_train, alpha=1.0, beta=1.0)
            else:
                kernel_output = kernel(X_train, X_train)
                kernel_output_test = kernel(X_test, X_train)

            print(kernel_output.shape, kernel_output_test.shape)

            # Define the model, regularization parameters, and optimizer
            dense = tf.keras.layers.Dense(units=1, kernel_regularizer=regularizers.l1(0.1))
            # loss = tf.losses.mean_absolute_error(y_train, output)
            optimizer = tf.optimizers.Adam()

            # training loop
            for epoch in range(1000):
                with tf.GradientTape() as tape:
                    logits = dense(tf.reshape(kernel_output, (-1, kernel_output.shape[1]))) #logits are just the output of dense layer, generally prior to activation layer
                    loss_value = tf.losses.mean_absolute_error(y_train, logits)
                grads = tape.gradient(loss_value, dense.trainable_variables)
                optimizer.apply_gradients(zip(grads, dense.trainable_variables))

            # Evaluate the model on the test set
            test_output = dense(kernel_output_test) #Runs the now trained dense model on the test set
            print(test_output, kernel_name, np.sum(np.isnan(test_output)))
            test_mae = tf.losses.mean_absolute_error(y_test, test_output)
            mae_list.append(test_mae)

    return mae_list

mae_list = leave_one_out_cross_validation(data_df, kernels, kernel_names)
print("MAE: ", mae_list)


In [ ]:
print("MAE: ", mae_list)

In [ ]:
import tensorflow as tf

# Define RBF kernel function
def rbf_kernel(X1, X2, gamma=1):
    """RBF (Radial Basis Function) kernel function"""
    sq_dists = tf.reduce_sum(X1**2, 1, keepdims=True) + tf.reduce_sum(X2**2, 1, keepdims=True) - 2 * tf.matmul(X1, X2, transpose_b=True)
    return tf.exp(-gamma * sq_dists)

# Define input data
X1 = tf.constant([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]], dtype=tf.float32)
X2 = tf.constant([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
                [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
                [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
                [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
                ], dtype=tf.float32)

# Pass input data through RBF kernel
kernel_output = rbf_kernel(X1, X2, gamma=0.0001)

# Pass kernel output through dense layer
dense = tf.keras.layers.Dense(units=1)
output = dense(kernel_output)

# Print prediction
print(output)
